In [1]:
import rpy2
%load_ext rpy2.ipython

import numpy as np
from glmnet.glmnet import RegGLM
rng = np.random.default_rng(0)

In [2]:
n, p = 1000, 50
X = rng.standard_normal((n, p))
beta = np.zeros(p)
beta[:2] = [1,2]
y = rng.standard_normal(n) + X @ beta
W = np.ones(n) # rng.uniform(1, 2, size=(n,))

In [3]:
%%R -i X,y,W,n,p -o coef_,intercept_
library(glmnet)
G = glmnet(X, y, standardize=FALSE, intercept=TRUE, weights=W)
B = predict(G, s=2 / sqrt(n), type='coef', exact=TRUE, x=X, y=y, weights=W)
coef_ = B[2:(p+1)]
intercept_ = B[1]

Loading required package: Matrix
Loaded glmnet 4.1-7


In [4]:
coef_

array([ 8.96664353e-01,  1.90821670e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -8.75543063e-04,  0.00000000e+00,  0.00000000e+00,  1.84666823e-02,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])

In [7]:
G = RegGLM(lambda_val=2 / np.sqrt(n), standardize=False, fit_intercept=True)
G.fit(X, y, sample_weight=W)
G.coef_

array([ 8.96664354e-01,  1.90821670e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -8.75543163e-04,  0.00000000e+00,  0.00000000e+00,  1.84666824e-02,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])

In [8]:
np.testing.assert_allclose(G.intercept_, intercept_)

In [11]:
np.testing.assert_allclose(G.coef_, coef_, rtol=1e-5, atol=1e-5)

In [12]:
from sklearn.model_selection import cross_validate
cross_validate(G, X, y, cv=5)

{'fit_time': array([0.01464605, 0.00067592, 0.00067091, 0.0005672 , 0.00055385]),
 'score_time': array([2.57492065e-05, 2.09808350e-05, 2.02655792e-05, 1.97887421e-05,
        1.90734863e-05]),
 'test_score': array([-114.34899496, -102.4599904 ,  -94.61822674,  -84.92798059,
        -115.13325371])}